In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    break
    
for i in ds_test.batch(60000):
    b = i
    break

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

#dtrain = xgb.DMatrix(xtrain,label=ytrain)
#dtest = xgb.DMatrix(xtest,label=ytest)

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = lgb.train(param,
                      train_data,
                      num_boost_round=20,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['multi_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'objective': hp.choice('objective',['multiclass']),
    'metric':hp.choice('metric',['multi_error']),
    'num_class':hp.choice('num_class',[10]),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.724947277751181, 'feature_fraction': 0.8277559755538413, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.704200807704493, 'metric': 'multi_error', 'min_data_in_leaf': 81.0, 'min_sum_hessian_in_leaf': 5.031669975211453e-07, 'num_class': 10, 'num_leaves': 2.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.4515                      
[2]	valid_0's multi_error: 0.3602                      
[3]	valid_0's multi_error: 0.2943                      
[4]	valid_0's multi_error: 0.2534                      
[5]	valid_0's multi_error: 0.2172                      
[6]	valid_0's multi_error: 0.2006                      
[7]	valid_0's multi_error: 0.1868                      
[8]	valid_0's multi_error: 0.1731                      
[9]	valid_0's multi_error: 0.1604                      
[10]	valid_0's multi_error: 0.1527                     
[11]	valid_0's multi_error: 0.1475                     
[12]	valid_0's multi_error: 0.1436                     
[13]	valid_0's m

[16]	valid_0's multi_error: 0.2668                                     
[17]	valid_0's multi_error: 0.2617                                     
[18]	valid_0's multi_error: 0.2528                                     
[19]	valid_0's multi_error: 0.2508                                     
[20]	valid_0's multi_error: 0.2453                                     
{'bagging_fraction': 0.8024323311366416, 'feature_fraction': 0.9259739706550074, 'lambda_l1': 0, 'lambda_l2': 0.017399654023025398, 'learning_rate': 0.008634297441452152, 'metric': 'multi_error', 'min_data_in_leaf': 11.0, 'min_sum_hessian_in_leaf': 0.998439428180779, 'num_class': 10, 'num_leaves': 1017.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.8865                                    
[2]	valid_0's multi_error: 0.8019                                    
[3]	valid_0's multi_error: 0.3793                                    
[4]	valid_0's multi_error: 0.2247                                    
[5]	valid_0's multi_error:

[6]	valid_0's multi_error: 0.5456                                      
[7]	valid_0's multi_error: 0.4331                                      
[8]	valid_0's multi_error: 0.3686                                      
[9]	valid_0's multi_error: 0.3147                                      
[10]	valid_0's multi_error: 0.2731                                     
[11]	valid_0's multi_error: 0.2419                                     
[12]	valid_0's multi_error: 0.2183                                     
[13]	valid_0's multi_error: 0.2026                                     
[14]	valid_0's multi_error: 0.1895                                     
[15]	valid_0's multi_error: 0.1797                                     
[16]	valid_0's multi_error: 0.1684                                     
[17]	valid_0's multi_error: 0.1589                                     
[18]	valid_0's multi_error: 0.1512                                     
[19]	valid_0's multi_error: 0.1469                              

[20]	valid_0's multi_error: 0.0502                                    
{'bagging_fraction': 0.7678258938825182, 'feature_fraction': 0.8284622916149957, 'lambda_l1': 2.7745211504500804, 'lambda_l2': 0, 'learning_rate': 0.04401152635780697, 'metric': 'multi_error', 'min_data_in_leaf': 13.0, 'min_sum_hessian_in_leaf': 4.3328897239267014e-07, 'num_class': 10, 'num_leaves': 737.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.157                                      
[2]	valid_0's multi_error: 0.1006                                     
[3]	valid_0's multi_error: 0.0823                                     
[4]	valid_0's multi_error: 0.074                                      
[5]	valid_0's multi_error: 0.0674                                     
[6]	valid_0's multi_error: 0.0642                                     
[7]	valid_0's multi_error: 0.0612                                     
[8]	valid_0's multi_error: 0.0585                                     
[9]	valid_0's multi_error:

[11]	valid_0's multi_error: 0.1217                                    
[12]	valid_0's multi_error: 0.1203                                    
[13]	valid_0's multi_error: 0.12                                      
[14]	valid_0's multi_error: 0.1191                                    
[15]	valid_0's multi_error: 0.1184                                    
[16]	valid_0's multi_error: 0.1152                                    
[17]	valid_0's multi_error: 0.1141                                    
[18]	valid_0's multi_error: 0.111                                     
[19]	valid_0's multi_error: 0.1093                                    
[20]	valid_0's multi_error: 0.1071                                    
{'bagging_fraction': 0.8640595597830358, 'feature_fraction': 0.8138185821834523, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.1640816159113986, 'metric': 'multi_error', 'min_data_in_leaf': 2.0, 'min_sum_hessian_in_leaf': 2.962852277058632, 'num_class': 10, 'num_leaves': 4.0, 'objecti

...

{'bagging_fraction': 0.5552461630816734, 'feature_fraction': 0.7856968865455767, 'lambda_l1': 0.0031335017284089555, 'lambda_l2': 0, 'learning_rate': 0.4630056541989954, 'metric': 'multi_error', 'min_data_in_leaf': 24.0, 'min_sum_hessian_in_leaf': 3.424764056187364e-05, 'num_class': 10, 'num_leaves': 227.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.1013                                       
[2]	valid_0's multi_error: 0.0711                                       
[3]	valid_0's multi_error: 0.0594                                       
[4]	valid_0's multi_error: 0.0515                                       
[5]	valid_0's multi_error: 0.0476                                       
[6]	valid_0's multi_error: 0.0436                                       
[7]	valid_0's multi_error: 0.0407                                       
[8]	valid_0's multi_error: 0.0402                                       
[9]	valid_0's multi_error: 0.0384                                       
[10]	va

[11]	valid_0's multi_error: 0.0367                                      
[12]	valid_0's multi_error: 0.0357                                      
[13]	valid_0's multi_error: 0.0346                                      
[14]	valid_0's multi_error: 0.0339                                      
[15]	valid_0's multi_error: 0.0335                                      
[16]	valid_0's multi_error: 0.0332                                      
[17]	valid_0's multi_error: 0.0328                                      
[18]	valid_0's multi_error: 0.0321                                      
[19]	valid_0's multi_error: 0.0323                                      
[20]	valid_0's multi_error: 0.0318                                      
{'bagging_fraction': 0.6440173309674924, 'feature_fraction': 0.941307783797407, 'lambda_l1': 0.6652892220198555, 'lambda_l2': 0, 'learning_rate': 0.3124815945524292, 'metric': 'multi_error', 'min_data_in_leaf': 121.0, 'min_sum_hessian_in_leaf': 4.794645107309883e-06, 'n

[1]	valid_0's multi_error: 0.8865                                        
[2]	valid_0's multi_error: 0.8865                                        
[3]	valid_0's multi_error: 0.8865                                        
[4]	valid_0's multi_error: 0.8865                                        
[5]	valid_0's multi_error: 0.8865                                        
[6]	valid_0's multi_error: 0.8123                                        
[7]	valid_0's multi_error: 0.762                                         
[8]	valid_0's multi_error: 0.7405                                        
[9]	valid_0's multi_error: 0.5378                                        
[10]	valid_0's multi_error: 0.3824                                       
[11]	valid_0's multi_error: 0.3119                                       
[12]	valid_0's multi_error: 0.2738                                       
[13]	valid_0's multi_error: 0.249                                        
[14]	valid_0's multi_error: 0.2309    

[14]	valid_0's multi_error: 0.0367                                       
[15]	valid_0's multi_error: 0.0346                                       
[16]	valid_0's multi_error: 0.0339                                       
[17]	valid_0's multi_error: 0.0323                                       
[18]	valid_0's multi_error: 0.0324                                       
[19]	valid_0's multi_error: 0.0313                                       
[20]	valid_0's multi_error: 0.0295                                       
{'bagging_fraction': 0.5318830211847344, 'feature_fraction': 0.9498031940497317, 'lambda_l1': 0.04994040296270974, 'lambda_l2': 0.00021802483862312487, 'learning_rate': 0.2070089520502347, 'metric': 'multi_error', 'min_data_in_leaf': 4.0, 'min_sum_hessian_in_leaf': 0.38762516881896486, 'num_class': 10, 'num_leaves': 75.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.1367                                       
[2]	valid_0's multi_error: 0.0873                         

[3]	valid_0's multi_error: 0.0657                                       
[4]	valid_0's multi_error: 0.0599                                       
[5]	valid_0's multi_error: 0.058                                        
[6]	valid_0's multi_error: 0.055                                        
[7]	valid_0's multi_error: 0.0527                                       
[8]	valid_0's multi_error: 0.0515                                       
[9]	valid_0's multi_error: 0.0499                                       
[10]	valid_0's multi_error: 0.0486                                      
[11]	valid_0's multi_error: 0.0473                                      
[12]	valid_0's multi_error: 0.0465                                      
[13]	valid_0's multi_error: 0.0448                                      
[14]	valid_0's multi_error: 0.0435                                      
[15]	valid_0's multi_error: 0.0428                                      
[16]	valid_0's multi_error: 0.0411                 

In [4]:
print(best_val)
print(space_eval(spc, best))

0.0228
{'bagging_fraction': 0.574133886938671, 'feature_fraction': 0.7908243957709954, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.9864751824264205, 'metric': 'multi_error', 'min_data_in_leaf': 91.0, 'min_sum_hessian_in_leaf': 1.1249585585896708e-05, 'num_class': 10, 'num_leaves': 370.0, 'objective': 'multiclass'}
